In [ ]:
!pip install gigachat
!pip install pypdf
!pip install -U langchain-gigachat langchain-chroma langchain-core langchain-text-splitters langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 10.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-chroma to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 126.3 MB/s eta 0:00:00

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="huggingface_hub")

# GigaChat RAG example

Let us build a RAG system, which uses GigaChat.

## GigaChat basic usage exmaple


Fist, we need to initialize a GigaChat model.

In [ ]:
from gigachat import GigaChat
token = 'YOUR_TOKEN'
print(len(token))
giga = GigaChat(
   credentials=token,
   scope="GIGACHAT_API_PERS",
   model="GigaChat-2-Max",
   verify_ssl_certs=False
)

100


Basic usage.

In [ ]:
text = "Какие риски при использовании LLM?"
response = giga.chat(text)
print(response.choices[0].message.content)

Использование больших языковых моделей (LLM), таких как GPT-4, BERT, PaLM и другие подобные модели, связано с рядом рисков и проблем. Вот наиболее важные из них:

### 1. **Генерация недостоверной информации («галлюцинация»)**
   Языковые модели иногда генерируют правдоподобную, но ложную информацию, поскольку работают исключительно на основе статистического анализа текста и не имеют встроенных механизмов проверки фактов. Это особенно опасно в сферах медицины, права, финансов и науки, где ошибка может привести к серьезным последствиям.
   
### 2. **Непредсказуемость поведения модели**
   Несмотря на наличие предварительной настройки и фильтраций, модели могут неожиданно выдавать некорректный или неуместный контент: шутки на деликатные темы, дискриминационные высказывания, сексистские или расистские утверждения. Особенно рискованно использование моделей в публичных сервисах без должного контроля и фильтрации.

### 3. **Утечка конфиденциальной информации**
   Если пользователи вводят личн

In [ ]:
text = "Что такое задача понижения размерности?"
response = giga.chat(text)
print(response.choices[0].message.content)

Задача понижения размерности (Dimensionality Reduction) — это метод обработки многомерных данных, направленный на уменьшение количества признаков (размерности пространства), сохраняя наиболее значимую информацию. Она помогает упростить структуру данных, улучшить интерпретируемость моделей машинного обучения и уменьшить вычислительные затраты.

## Основные цели снижения размерности:
- **Уменьшение избыточности**: устранение дублирования информации между признаками.
- **Повышение производительности алгоритмов**: снижение временных затрат на обучение и работу модели.
- **Снижение вероятности переобучения** («проклятие размерности»).
- **Облегчение визуализации данных**, особенно когда исходное пространство имеет высокую размерность.
  
## Методы снижения размерности:
### Линейные методы:
1. **PCA (Principal Component Analysis)** — один из популярных методов, который находит новые оси координат, максимизирующие дисперсию данных.
   
   *Пример:* Представьте набор изображений рукописных циф

### RAG

For GigaChat we will use `langchain_gigachat` instead of the vanilla `langchain`, which is the official integration between GigaChat and LangChain.

It provides LangChain-compatible classes so you can use GigaChat in:
* Retrieval-Augmented Generation (RAG)
* LCEL (LangChain Expression Language)
* Agents & Tools
* Chains (like RetrievalQA, LLMChain)
* Vector stores and document QA

In [ ]:
# Imports
import os
from glob import glob
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.embeddings import Embeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA

from sentence_transformers import SentenceTransformer


## Russian Embedder
To work with Russian documents we need a Russian embedder model. Today, we will use [FRIDA](https://huggingface.co/ai-forever/FRIDA), which is a good Russian-English embedder.

FRIDA was trained for several embedding tasks with special prefixes, which indicate the embedding type.

For retrieval the following prefixes are used:
* `search_query`: for queries
* `search_document`: for documents


In [ ]:
import torch
class FridaEmbeddings(Embeddings):

    def __init__(self, model_name = "ai-forever/FRIDA"):
        self.model = SentenceTransformer(model_name)

        # Device handling
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = self.model.to(device)
        self.device = device

    def embed_documents(self, texts):
        texts = [f"search_document: {t}" for t in texts]
        # convert_to_numpy=True возвращает np.ndarray; tolist() -> list[list[float]]
        return self.model.encode(texts, normalize_embeddings=True, convert_to_numpy=True).tolist()

    def embed_query(self, text):
        return self.model.encode(f"search_query: {text}",
                                 normalize_embeddings=True, convert_to_numpy=True).tolist()


In [ ]:
# select which embeddings we want to use
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "ai-forever/FRIDA"
embeddings = FridaEmbeddings(model_name=embedding_model_name)

## Documents



In [ ]:
! wget https://raw.githubusercontent.com/esokolov/ml-course-hse/master/2025-fall/lecture-notes/lecture01-intro.pdf -O lecture01-intro.pdf


--2025-10-19 14:11:20--  https://raw.githubusercontent.com/esokolov/ml-course-hse/master/2025-fall/lecture-notes/lecture01-intro.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 134730 (132K) [application/octet-stream]
Saving to: ‘lecture01-intro.pdf’

lecture01-intro.pdf 100%[===================>] 131.57K  --.-KB/s    in 0.01s   

2025-10-19 14:11:20 (9.30 MB/s) - ‘lecture01-intro.pdf’ saved [134730/134730]



RAG-system is build on the first [ML-lecture notes](https://github.com/esokolov/ml-course-hse/blob/master/2025-fall/lecture-notes/lecture01-intro.pdf) from HSE ML-course.

In [ ]:
# load document
loader = PyPDFLoader("lecture01-intro.pdf")
documents = loader.load()

### Vector database

`RecursiveCharacterTextSplitter` is a text-splitting utility from LangChain designed to divide long documents into manageable chunks while preserving as much semantic meaning and context as possible.

**What it does?**

It takes long text and splits it into smaller pieces (chunks) using a recursive hierarchy of separators (like paragraphs → sentences → words → characters).

**This ensures:**

* Chunks are within a maximum token or character limit.
* Text is split at logical boundaries whenever possible.
* Only if no larger boundaries exist, it splits more aggressively.

**Why it’s useful**

Language models have context length limits (like 4k, 8k, 32k tokens).
To use long texts in RAG, summarization, embedding, or indexing, you must split them.

RecursiveCharacterTextSplitter is preferred because:
* Smarter than simple character splitting
* Keeps sentences and paragraphs intact when possible
* Handles multilingual and noisy text well
* Supports overlaps to preserve context across chunks

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=120)
docs = splitter.split_documents(documents)

In [ ]:
# create the vectorestore to use as the index
db = Chroma.from_documents(docs, embeddings)

## Retriever

In [ ]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

## Gigachat

**Warning!** To use GigaChat for RAG we need a langchain-compatible variant:

`from langchain_gigachat import GigaChat as LC_GigaChat`

In [ ]:
from langchain_gigachat import GigaChat as LC_GigaChat
llm = LC_GigaChat(
    credentials=token,
    scope="GIGACHAT_API_PERS",
    verify_ssl_certs=False,           # для тестовой среды
    model="GigaChat-2-Max",                 # GigaChat-2 / -2-Pro / -2-Max — по желанию
)

### Creating a RetrievalQA Pipeline

The RetrievalQA chain connects a large language model (LLM) with your retriever interface.


You can also specify the chain type as one of four options:
* stuff
* map_reduce
* refine
* map_rerank

1. The default type, **stuff**, includes all text from the retrieved documents directly in the prompt.
2. The **map_reduce** type splits the texts into groups, asks the LLM the question for each group separately, and then produces a final answer based on all group responses.
3. The **refine** type splits the texts into chunks, presents the first chunk to the LLM, then sends the LLM’s response together with the next chunk, gradually refining the answer as it processes all batches.
4. The **map_rerank** type divides the texts into chunks, asks the LLM to evaluate how well each chunk answers the question, and then determines the final answer based on the highest-scoring responses.

In [ ]:
# Create a RetrievalQA chain for answering questions
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # Specify the language model to use
    chain_type="map_reduce",  # Use a map-reduce chain type for efficient large-scale queries
    retriever=retriever,  # Define the retriever to handle document searches
    return_source_documents=True,  # Return source documents along with the generated answers
    verbose=True  # Enable detailed logging for better debugging and monitoring
)

In [ ]:
question = "Что такое задача понижения размерности?"
result = qa_chain.invoke(question)
print(result['result'])



> Entering new RetrievalQA chain...

> Finished chain.
Задача понижения размерности заключается в создании нового набора признаков, который включает меньшее количество характеристик по сравнению с исходным набором, однако позволяет решать задачу не хуже (либо с минимальными потерями качества решения, либо даже лучше), чем на основе исходного множества признаков. В эту категорию также входит построение латентных моделей, которые описывают процесс формирования данных через небольшой набор скрытых переменных.


In [ ]:

for document in result["source_documents"]:
    print(document.page_content)

4
4. Понижение размерности /emdash.cyr задача генерации таких новых признаков, что их
меньше, чем исходных, но при этом с их помощью задача решаетс я не хуже (или
с небольшими потерями качества, или лучше /emdash.cyr зависит от постановки). К этой
же категории относится задача построения латентных моделе й, где требуется
описать процесс генерации данных с помощью некоторого (как правило, неболь-
шого) набора скрытых переменных. Примерами являются задач и тематическо-
3
окрестности, количество школ, магазинов, заправок, торго вых центров, банков по-
близости). Разработка признаков (feature engineering) дл я любой задачи является
одним из самых сложных и самых важных этапов анализа данных.
Описанная задача является примером задачи обучения с учителем (supervised
learning), а более конкретно задачей регрессии /emdash.cyr именно так называются задачи с
вещественной целевой переменной. Перечислим несколько др угих видов задач обу-
чения с учителем:
мы будем обсуждать на следующих занятиях. 